In [1]:
import pandas as pd
import numpy as np
import shutil
import os
import sys
from datetime import datetime as dt

In [2]:
# read in upd_gen csv
upd_gen_df = pd.read_csv('a_upd_generator_df.csv')
smr_upd_gen_df = upd_gen_df.copy()

In [3]:
# load in SMR data
smr_foak_cost_df = pd.read_csv(os.path.join('..', 'data', 'SMR_data', 'FOAK_Costs.csv'))

In [4]:
smr_foak_cost_df

,Sites,Source,Type,Power in MWe,Power in MWt,Thermal Transfer Efficiency,MSL in MWe,MSL_turb in MWe,Thermal Efficiency,MDT in hours,...,OS Learning_Low,Mod Learning_Low,OS Learning_Med,Mod Learning_Med,OS Learning_High,Mod Learning_High,OS Learning_High_Prop,Mod Learning_High_Prop,Unnamed: 36,Unnamed: 37
0,HTGR,NaN,HTGR,164.0,350,0.99,32.8,16.40,0.468571,20,...,-0.291,0,0.05,0.16,0.255,0.16,0.164384,0.526027,NaN,0.239967
1,Microreactor,https://www.energy.gov/sites/prod/files/2015/1...,HTGR,6.7,20,0.99,2.7,0.67,0.335000,20,...,-0.310,0,0.05,0.16,0.261,0.16,0.111111,0.355556,NaN,0.239444
2,PBR-HTGR,NaN,HTGR,80.0,200,0.99,32.0,8.00,0.400000,20,...,-0.484,0,0.05,0.16,0.312,0.16,0.153846,0.492308,NaN,0.240831
3,iPWR_Pack,NaN,PWR,462.0,1500,0.99,15.4,7.70,0.308000,20,...,-0.311,0,0.05,0.16,0.261,0.16,0.151899,0.486076,NaN,0.238785


In [5]:
# list SMR types
smr_types = [
    'HTGR',
    'Microreactor',
    'PBR-HTGR',
    'iPWR_Pack',
]

# isolate SMR rows
smr_rows = smr_upd_gen_df[smr_upd_gen_df['Resource'].isin(smr_types)]


In [6]:
# the data that should exist for RICE is:
reqd_thermal_data = [
                  'Resource',
                #   'Zone',
                #   'Model',
                #   'New_Build',
                #   'Can_Retire',
                #   'Existing_Cap_MW',
                #   'Max_Cap_MW',
                #   'Min_Cap_MW',
                  'Inv_Cost_per_MWyr',
                  'Fixed_OM_Cost_per_MWyr',
                  'Var_OM_Cost_per_MWh',
                  'Heat_Rate_MMBTU_per_MWh',
                  'Fuel',
                  'Cap_Size',
                  'Start_Cost_per_MW',
                  # 'Start_Fuel_MMBTU_per_MW',
                  'Up_Time',
                  'Down_Time',
                  'Ramp_Up_Percentage',
                  'Ramp_Dn_Percentage',
                  'Min_Power',
                #   'Reg_Max',
                #   'Reg_Min',
                #   'Reg_Cost',
                #   'Rsv_Cost',
                #   'region',
                #   'cluster',
                  ]

In [7]:
import textwrap

# determine missing data
missing_data = {}

for index, row in smr_rows.iterrows():
    missing_columns = [col for col in reqd_thermal_data if pd.isna(row[col])]
    if missing_columns:
        missing_data[row['Resource']] = missing_columns

for resource, columns in missing_data.items():
    print(resource)
    print(textwrap.fill(", ".join(columns), width=70))
    print()

HTGR
Inv_Cost_per_MWyr, Fixed_OM_Cost_per_MWyr, Var_OM_Cost_per_MWh,
Heat_Rate_MMBTU_per_MWh, Fuel, Cap_Size, Start_Cost_per_MW, Up_Time,
Down_Time, Ramp_Up_Percentage, Ramp_Dn_Percentage, Min_Power

Microreactor
Inv_Cost_per_MWyr, Fixed_OM_Cost_per_MWyr, Var_OM_Cost_per_MWh,
Heat_Rate_MMBTU_per_MWh, Fuel, Cap_Size, Start_Cost_per_MW, Up_Time,
Down_Time, Ramp_Up_Percentage, Ramp_Dn_Percentage, Min_Power

PBR-HTGR
Inv_Cost_per_MWyr, Fixed_OM_Cost_per_MWyr, Var_OM_Cost_per_MWh,
Heat_Rate_MMBTU_per_MWh, Fuel, Cap_Size, Start_Cost_per_MW, Up_Time,
Down_Time, Ramp_Up_Percentage, Ramp_Dn_Percentage, Min_Power

iPWR_Pack
Inv_Cost_per_MWyr, Fixed_OM_Cost_per_MWyr, Var_OM_Cost_per_MWh,
Heat_Rate_MMBTU_per_MWh, Fuel, Cap_Size, Start_Cost_per_MW, Up_Time,
Down_Time, Ramp_Up_Percentage, Ramp_Dn_Percentage, Min_Power



In [8]:
# assumptions
gen_lifetime = 20

In [9]:
for index, row in smr_foak_cost_df.iterrows():
    # get the resource
    resource = row['Sites']

    Cap_Size = row['Power in MWe']

    Min_Power = row['MSL in MWe'] / Cap_Size

    Down_Time = row['MDT in hours']
    # assume up time is same as down time
    Up_Time = Down_Time

    Ramp_Dn_Percentage = row['Ramp Rate (fraction of capacity/hr)']

    # assume ramp up is same as ramp down
    Ramp_Up_Percentage = Ramp_Dn_Percentage

    Inv_Cost_per_MWyr = row['CAPEX $/kWe'] * 1000 / gen_lifetime

    Fixed_OM_Cost_per_MWyr = row['FOPEX $/kWe'] * 1000 / gen_lifetime

    Var_OM_Cost_per_MWh = row['VOM in $/MWh-e']

    # fuel Cost
    Start_Cost_per_MW = row['Startupfixedcost in $'] / Cap_Size

    Heat_Rate_MMBTU_per_MWh = 10.46 # hard coded from GenX

    Fuel = 'UM'

    # add data to smr_upd_gen_df
    smr_upd_gen_df.loc[smr_upd_gen_df['Resource'] == resource, 'Cap_Size'] = Cap_Size
    smr_upd_gen_df.loc[smr_upd_gen_df['Resource'] == resource, 'Min_Power'] = Min_Power
    smr_upd_gen_df.loc[smr_upd_gen_df['Resource'] == resource, 'Down_Time'] = Down_Time
    smr_upd_gen_df.loc[smr_upd_gen_df['Resource'] == resource, 'Up_Time'] = Up_Time
    smr_upd_gen_df.loc[smr_upd_gen_df['Resource'] == resource, 'Ramp_Dn_Percentage'] = Ramp_Dn_Percentage
    smr_upd_gen_df.loc[smr_upd_gen_df['Resource'] == resource, 'Ramp_Up_Percentage'] = Ramp_Up_Percentage
    smr_upd_gen_df.loc[smr_upd_gen_df['Resource'] == resource, 'Inv_Cost_per_MWyr'] = Inv_Cost_per_MWyr
    smr_upd_gen_df.loc[smr_upd_gen_df['Resource'] == resource, 'Fixed_OM_Cost_per_MWyr'] = Fixed_OM_Cost_per_MWyr
    smr_upd_gen_df.loc[smr_upd_gen_df['Resource'] == resource, 'Var_OM_Cost_per_MWh'] = Var_OM_Cost_per_MWh
    smr_upd_gen_df.loc[smr_upd_gen_df['Resource'] == resource, 'Heat_Rate_MMBTU_per_MWh'] = Heat_Rate_MMBTU_per_MWh
    smr_upd_gen_df.loc[smr_upd_gen_df['Resource'] == resource, 'Start_Cost_per_MW'] = Start_Cost_per_MW
    smr_upd_gen_df.loc[smr_upd_gen_df['Resource'] == resource, 'Fuel'] = Fuel


In [10]:
second_check_smr_df = smr_upd_gen_df[smr_upd_gen_df['Resource'].isin(smr_types)]

# check that no batteries are missing data
missing_data = {}

for index, row in second_check_smr_df.iterrows():
    missing_columns = [col for col in reqd_thermal_data if pd.isna(row[col])]
    if missing_columns:
        missing_data[row['Resource']] = missing_columns

if not missing_data:
    print("No SMRs are missing data.")
else:
    for resource, columns in missing_data.items():
        print(resource)
        print(textwrap.fill(", ".join(columns), width=70))
        print()

No SMRs are missing data.


In [11]:
# save to csv
smr_upd_gen_df.to_csv('a_upd_generator_df.csv', index=False)